In [ ]:
pip install gradio numpy rasterio

In [2]:
import gradio as gr
import numpy as np
import rasterio
from rasterio.errors import RasterioIOError

def create_false_color_composite(tiff_file: object) -> np.ndarray:
    """
    Generates a false-color composite image from a multi-band TIFF file.

    This function reads a satellite image, like one from Landsat 8/9, and uses
    bands 7 (SWIR 2), 3 (Green), and 2 (Blue) to create a false-color image.

    Args:
        tiff_file: A file object from the Gradio interface.

    Returns:
        A NumPy array representing the 8-bit false-color image.

    Raises:
        gr.Error: If the file is invalid, can't be read, or doesn't have enough bands.
    """
    if tiff_file is None:
        raise gr.Error("Please upload a TIFF file.")

    try:
        # Open the uploaded file using rasterio
        with rasterio.open(tiff_file.name) as src:
            # Check if the image has the required number of bands (at least 7)
            if src.count < 7:
                raise gr.Error(f"Input image has only {src.count} bands, but at least 7 are required.")

            # Read bands 7, 3, and 2. Rasterio bands are 1-indexed.
            # These are mapped to the Red, Green, and Blue channels of the output image.
            red = src.read(7)    # SWIR 2
            green = src.read(3)  # Green
            blue = src.read(2)   # Blue

            # Stack the bands into a 3D array (height, width, channels)
            false_color_composite = np.dstack((red, green, blue))

            # Normalize the image to the 0-255 range for display.
            min_val = false_color_composite.min()
            max_val = false_color_composite.max()

            # Avoid division by zero if the image data is flat
            if max_val == min_val:
                return np.zeros_like(false_color_composite, dtype=np.uint8)

            # Perform normalization and scale to 8-bit integer range [0, 255]
            normalized = (false_color_composite - min_val) / (max_val - min_val)
            image_uint8 = (normalized * 255).astype(np.uint8)

            return image_uint8

    except RasterioIOError:
        raise gr.Error("Failed to read the file. Please ensure it's a valid GeoTIFF file.")
    except Exception as e:
        raise gr.Error(f"An unexpected error occurred: {e}")

# --- Gradio Interface ---
iface = gr.Interface(
    fn=create_false_color_composite,
    inputs=gr.File(label="Upload Satellite TIFF Image"),
    outputs=gr.Image(type="numpy", label="False-Color Composite (Bands 7-3-2)"),
    title="🛰️ Satellite False-Color Composite Generator",
    description="Upload a multi-band satellite image (e.g., Landsat 8/9) in TIFF format. "
                "This tool generates a false-color composite using bands 7 (SWIR 2), 3 (Green), and 2 (Blue).",
    article="<p style='text-align: center;'>This band combination is often used for geology and soil analysis. "
            "In this view, urban areas typically appear purple, vegetation is green/brown, and water is dark blue/black.</p>",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d4c16db8320c15a3c7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import gradio as gr
import numpy as np
import rasterio
from rasterio.errors import RasterioIOError

def highlight_water_bodies(tiff_file: object, ndwi_threshold: float) -> np.ndarray:
    """
    Identifies water bodies in a satellite image using the Normalized Difference
    Water Index (NDWI) and highlights them on a natural color background.

    This function requires bands for Red, Green, Blue, and Near-Infrared (NIR).
    For Landsat 8/9, these are bands 4, 3, 2, and 5, respectively.

    Args:
        tiff_file: A file object from the Gradio interface.
        ndwi_threshold: A float value (typically 0.0 to 0.3) used as a
                        cutoff to classify pixels as water.

    Returns:
        A NumPy array representing the image with water bodies highlighted in blue.

    Raises:
        gr.Error: If the file is invalid or doesn't have the required bands.
    """
    if tiff_file is None:
        raise gr.Error("Please upload a TIFF file.")

    try:
        with rasterio.open(tiff_file.name) as src:
            # Check if the image has enough bands for NDWI and Natural Color
            # We need Blue (2), Green (3), Red (4), and NIR (5) for Landsat 8/9
            if src.count < 5:
                raise gr.Error(f"Input image has only {src.count} bands. "
                             "At least 5 bands are required for this analysis.")

            # --- Read Bands ---
            # For Natural Color background (Red, Green, Blue)
            red = src.read(4).astype(np.float32)
            green = src.read(3).astype(np.float32)
            blue = src.read(2).astype(np.float32)

            # For NDWI calculation (Green, NIR)
            nir = src.read(5).astype(np.float32)

            # --- Calculate NDWI ---
            # NDWI = (Green - NIR) / (Green + NIR)
            # Suppress warnings for division by zero and handle it by outputting 0
            denominator = np.add(green, nir)
            ndwi = np.divide(green - nir, denominator,
                             out=np.zeros_like(denominator, dtype=np.float32),
                             where=(denominator != 0))

            # Create a boolean mask for water based on the threshold
            water_mask = ndwi > ndwi_threshold

            # --- Create Visualization ---
            # Function to stretch each band to its 2nd and 98th percentile for better contrast
            def normalize_band(band):
                p2, p98 = np.percentile(band[band > 0], (2, 98)) # Use non-zero values for stats
                # Clip values to the percentile range
                stretched = np.clip(band, p2, p98)
                # Scale to 0-1 range
                scaled = (stretched - p2) / (p98 - p2)
                return scaled

            # Create a 3-band natural color image
            natural_color_image = np.dstack((normalize_band(red),
                                             normalize_band(green),
                                             normalize_band(blue)))

            # Convert to 8-bit for display
            background_image = (natural_color_image * 255).astype(np.uint8)

            # Apply the water mask, coloring water pixels bright blue
            # This creates a copy of the array to avoid changing the original
            output_image = background_image.copy()
            output_image[water_mask] = [60, 130, 255] # A distinct shade of blue

            return output_image

    except RasterioIOError:
        raise gr.Error("Failed to read the file. Please ensure it's a valid GeoTIFF file.")
    except Exception as e:
        raise gr.Error(f"An unexpected error occurred: {e}")

# --- Gradio Interface ---
iface = gr.Interface(
    fn=highlight_water_bodies,
    inputs=[
        gr.File(label="Upload Satellite TIFF Image"),
        gr.Slider(minimum=-0.5, maximum=0.8, value=0.2, step=0.05,
                  label="NDWI Threshold",
                  info="Adjust to fine-tune water detection. Higher values are more strict.")
    ],
    outputs=gr.Image(type="numpy", label="Water Bodies Highlighted"),
    title="💧 Water Body Detection from Satellite Imagery",
    description="Upload a multi-band satellite image (e.g., Landsat 8/9) to identify and highlight water bodies. "
                "The tool uses the Normalized Difference Water Index (NDWI).",
    article="<p style='text-align: center;'>The analysis uses a natural color image as a background and overlays water bodies (lakes, rivers, coasts) in bright blue. "
            "The <b>NDWI Threshold</b> can be adjusted to reduce noise from shadows or include turbid water.</p>",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33f0b15db0b84c9554.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
